<a href="https://colab.research.google.com/github/FritzAriza/TDG_Ariza_Roman/blob/main/convertidor_keras_h.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!sudo apt-get install xxd

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
xxd is already the newest version (2:8.2.3995-1ubuntu2.22).
xxd set to manually installed.
0 upgraded, 0 newly installed, 0 to remove and 18 not upgraded.


In [2]:
import os
import shutil
import pathlib
import tensorflow as tf

In [3]:
# Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Conversión a TensorFlow Lite con cuantización de enteros
def convert_to_tflite(model_path, drive_path):
  best_model = tf.keras.models.load_model(model_path)

  # Función para crear un conjunto de datos representativo
  def representative_dataset():
    # Cargar un conjunto de imágenes de espectrogramas de prueba
    test_directory = '/content/drive/MyDrive/dataset/data/test/'  # Asegúrate de que esta ruta sea correcta
    test_ds = tf.keras.preprocessing.image_dataset_from_directory(
        test_directory, labels='inferred', label_mode='int', image_size=(32, 32), shuffle=False, batch_size=32, color_mode='grayscale'
    )

    # Seleccionar un pequeño subconjunto de datos para la cuantización
    for image, _ in test_ds.take(100):  # Tomar 100 imágenes para el conjunto representativo
      yield [image]

  # Convertir a TensorFlow Lite con optimización para tamaño (cuantización)
  converter = tf.lite.TFLiteConverter.from_keras_model(best_model)
  converter.optimizations = [tf.lite.Optimize.DEFAULT]  # Optimización para tamaño
  converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]  # Cuantización a enteros
  converter.inference_input_type = tf.int8  # Tipo de entrada: entero de 8 bits
  converter.inference_output_type = tf.int8  # Tipo de salida: entero de 8 bits
  converter.representative_dataset = representative_dataset  # Establecer el conjunto de datos representativo
  tflite_model = converter.convert()

  # Create the directory if it does not exist:
  os.makedirs(os.path.dirname(drive_path), exist_ok=True)

  with open(drive_path, "wb") as f:
      f.write(tflite_model)

  print(f"Modelo convertido y guardado como mejor_modelo.tflite en {drive_path}")

In [5]:
# Cargar el modelo .keras y como se guardara el .tflite
model_path = './mejor_modelo.keras'
drive_path = './mejor_modelo.tflite'

# Conversión a TensorFlow Lite
convert_to_tflite(model_path, drive_path)

Saved artifact at '/tmp/tmphh88o67a'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 32, 32, 1), dtype=tf.float32, name='input_layer')
Output Type:
  TensorSpec(shape=(None, 6), dtype=tf.float32, name=None)
Captures:
  138906128136528: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138906128137680: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138906128133648: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138906128137104: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138906126091280: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138906126092048: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138906126092816: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138906126093392: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138906126092624: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138906126095120: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138906126093776: 

/usr/local/lib/python3.11/dist-packages/tensorflow/lite/python/convert.py:997: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(


Found 234 files belonging to 6 classes.
Modelo convertido y guardado como mejor_modelo.tflite en ./mejor_modelo.tflite


In [7]:
interpreter = tf.lite.Interpreter(model_path=drive_path)
interpreter.allocate_tensors()

In [8]:
tensor_details = interpreter.get_tensor_details()

In [12]:
print("Resumen del modelo TFLite:")
print("="*50)
for i, tensor in enumerate(tensor_details):
    print(f"Tensor {i}: {tensor['name']}")
    print(f"  - Índice: {tensor['index']}")
    print(f"  - Forma: {tensor['shape']}")
    print(f"  - Tipo: {tensor['dtype']}\n")

Resumen del modelo TFLite:
Tensor 0: serving_default_input_layer:0
  - Índice: 0
  - Forma: [ 1 32 32  1]
  - Tipo: <class 'numpy.int8'>

Tensor 1: arith.constant
  - Índice: 1
  - Forma: [2]
  - Tipo: <class 'numpy.int32'>

Tensor 2: tfl.pseudo_qconst
  - Índice: 2
  - Forma: [6]
  - Tipo: <class 'numpy.int32'>

Tensor 3: tfl.pseudo_qconst1
  - Índice: 3
  - Forma: [ 6 32]
  - Tipo: <class 'numpy.int8'>

Tensor 4: tfl.pseudo_qconst2
  - Índice: 4
  - Forma: [32]
  - Tipo: <class 'numpy.int32'>

Tensor 5: tfl.pseudo_qconst3
  - Índice: 5
  - Forma: [32 64]
  - Tipo: <class 'numpy.int8'>

Tensor 6: tfl.pseudo_qconst4
  - Índice: 6
  - Forma: [64]
  - Tipo: <class 'numpy.int32'>

Tensor 7: tfl.pseudo_qconst5
  - Índice: 7
  - Forma: [ 64 128]
  - Tipo: <class 'numpy.int8'>

Tensor 8: tfl.pseudo_qconst6
  - Índice: 8
  - Forma: [128]
  - Tipo: <class 'numpy.int32'>

Tensor 9: tfl.pseudo_qconst7
  - Índice: 9
  - Forma: [ 128 1024]
  - Tipo: <class 'numpy.int8'>

Tensor 10: tfl.pseudo_qcon

In [14]:
tf.lite.experimental.Analyzer.analyze(drive_path)

=== ./mejor_modelo.tflite ===

Your TFLite model has '1' subgraph(s). In the subgraph description below,
T# represents the Tensor numbers. For example, in Subgraph#0, the CONV_2D op takes
tensor #0 and tensor #21 and tensor #20 as input and produces tensor #22 as output.

Subgraph#0 main(T#0) -> [T#36]
  Op#0 CONV_2D(T#0, T#21, T#20[-58, -12, -30, -63, -49, ...]) -> [T#22]
  Op#1 CONV_2D(T#22, T#19, T#18[45, 17, 7, -17, -189, ...]) -> [T#23]
  Op#2 MAX_POOL_2D(T#23) -> [T#24]
  Op#3 CONV_2D(T#24, T#17, T#16[7, 234, 141, -178, 75, ...]) -> [T#25]
  Op#4 CONV_2D(T#25, T#15, T#14[-29, 25, 78, 20, -131, ...]) -> [T#26]
  Op#5 MAX_POOL_2D(T#26) -> [T#27]
  Op#6 CONV_2D(T#27, T#13, T#12[189, 249, 28, -12, -64, ...]) -> [T#28]
  Op#7 CONV_2D(T#28, T#11, T#10[184, 266, -65, -15, 206, ...]) -> [T#29]
  Op#8 MAX_POOL_2D(T#29) -> [T#30]
  Op#9 RESHAPE(T#30, T#1[-1, 1024]) -> [T#31]
  Op#10 FULLY_CONNECTED(T#31, T#9, T#8[66, -63, -143, -103, -30, ...]) -> [T#32]
  Op#11 FULLY_CONNECTED(T#32, T#7, 

In [18]:
# !xxd -i modelo.tflite > modelo.h
!xxd -i './mejor_modelo.tflite' > './mejor_modelo.h'

Cuando descarguen el .h recuerden definir las variables como:
```
const unsigned char modelo[] = {...};
const unsigned int modelo_len = 108432;
```
Porque al realizar la conversión no están definidas como const. Además, el nombre del modelo queda definido como la dirección completa que se haya indicado en la línea de conversión


In [ ]:
#Mostrar el archivo
!cat '/content/drive/MyDrive/dataset/modelo/mejor_modelo.h'

Se han truncado las últimas 5000 líneas del flujo de salida.
  0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00,
  0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00,
  0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00,
  0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00,
  0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00,
  0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00,
  0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00,
  0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00,
  0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00,
  0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00,
  0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00,
  0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00,
  0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0